In [2]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("deepseek.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("DEEPSEEK_API_KEY")
api_base = os.getenv("DEEPSEEK_API_BASE")
serp_api_key = os.getenv("SERPAPI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] =serp_api_key
# 为 DeepSeek 配置（可选）
DEEPSEEK_CONFIG = {
    #"model": "gpt-3.5-turbo",  # 或者使用 DeepSeek 兼容模型
    #"openai_api_key": os.environ["OPENAI_API_KEY"],
    # 如果使用 DeepSeek:
    "model": "deepseek-chat",
    "openai_api_base": os.environ["DEEPSEEK_API_BASE"],
    "openai_api_key": os.environ["DEEPSEEK_API_KEY"]
}

# 如何给agent正确的增加记忆

- 将memory插入到提示词模板中
<hr>

In [1]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain
from langchain.prompts import  MessagesPlaceholder


定义大模型

In [ ]:

llm=ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG
    )

/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_64512/1534123048.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(


构建agent可用工具

In [4]:
#构建一个搜索工具
search = SerpAPIWrapper()
#创建一个数学计算工具
llm_math_chain = LLMMathChain(
    llm=llm,
    verbose=True
)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
print(tools)

[Tool(name='Search', description='useful for when you need to answer questions about current events or the current state of the world', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='54593553f44d1d9c8c2e610beaf57d8de0fe6c25fce97628e591ef64f5ddad04', aiosession=None)>), Tool(name='Calculator', description='useful for when you need to answer questions about math', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the prob

/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_64512/3294897194.py:4: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  llm_math_chain = LLMMathChain(
/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_64512/3294897194.py:4: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  llm_math_chain = LLMMathChain(


增加memory组件

In [5]:
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_64512/1205651814.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


定义agent

In [6]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    memory=memory #记忆组件
    )

/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_64512/1161536941.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_chain = initialize_agent(


查看默认的agents prompt是什么样的

In [7]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.' additional_kwargs={} response_metadata={}
prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}') additional_kwargs={}
variable_name='agent_scratchpad'


In [ ]:
agent_chain.run("我叫什么名字？")



> Entering new AgentExecutor chain...
你好，李晨！很高兴认识你。如果你有任何问题或需要帮助的地方，请随时告诉我，我会尽力为你提供支持。祝你今天一切顺利！ 😊

> Finished chain.


> Entering new AgentExecutor chain...
目前我无法知道你的名字，因为你还没有告诉我！😊 但如果你愿意告诉我，我会记住并在接下来的对话中使用它。或者，你可以直接告诉我你想让我怎么称呼你～ 你希望我怎么称呼你呢？

> Finished chain.


'目前我无法知道你的名字，因为你还没有告诉我！😊 但如果你愿意告诉我，我会记住并在接下来的对话中使用它。或者，你可以直接告诉我你想让我怎么称呼你～ 你希望我怎么称呼你呢？'

需要使用agent_kwargs传递参数，将chat_history传入

In [12]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    agent_kwargs={
        "extra_prompt_messages":[MessagesPlaceholder(variable_name="chat_history"),MessagesPlaceholder(variable_name="agent_scratchpad")],
    },
    memory=memory #记忆组件
    )

查看重新渲染后的提示词模版

In [13]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}, response_metadata={}), MessagesPlaceholder(variable_name='chat_history'), MessagesPlaceholder(variable_name='agent_scratchpad'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.' additional_kwargs={} response_metadata={}
variable_name='chat_history'
variable_name='agent_scratchpad'


In [21]:
agent_chain.run("好厉害，刚才我们都聊了什么？")



> Entering new AgentExecutor chain...
作为一个文本交互的AI，我可以回顾我们的对话记录。到目前为止，我们的对话内容如下：

1. 您问我：“我叫什么名字？”
2. 我回答说，作为AI，我没有访问您个人信息的权限，所以我不知道您的名字，并提出如果您愿意告诉我，我会知道如何称呼您。

这就是我们目前为止的全部对话内容。如果您有其他问题或需要帮助，请随时告诉我！

> Finished chain.


'作为一个文本交互的AI，我可以回顾我们的对话记录。到目前为止，我们的对话内容如下：\n\n1. 您问我：“我叫什么名字？”\n2. 我回答说，作为AI，我没有访问您个人信息的权限，所以我不知道您的名字，并提出如果您愿意告诉我，我会知道如何称呼您。\n\n这就是我们目前为止的全部对话内容。如果您有其他问题或需要帮助，请随时告诉我！'